# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [4]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [5]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")
px.set_mapbox_access_token(map_box_api)

Python-dotenv could not parse statement starting at line 4
Python-dotenv could not parse statement starting at line 5
Python-dotenv could not parse statement starting at line 11


# Import Data

In [6]:
# Import the necessary CSVs to Pandas DataFrames
sfo_data_path = Path("./sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(sfo_data_path, index_col="year")

locations_path = Path("./neighborhoods_coordinates.csv")
df_neighborhood_locations = pd.read_csv(locations_path)

housing_units= sfo_data['housing_units'].groupby('year').mean()
san_fran_df = sfo_data.groupby(['year', 'neighborhood']).mean()


- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [7]:
# Define Panel Visualization Functions

def housing_units_per_year():

    figure_plot= plt.figure()
    housing_units= sfo_data['housing_units'].groupby('year').mean()
    min_units = min(housing_units)
    max_units = max(housing_units)
    std_units = housing_units.std()
    housing_plot = housing_units.plot.bar()
    housing_plot.set_xlabel("year")
    housing_plot.set_ylabel("Housing Units")
    housing_plot.set_title("Housing Units in San Fran 2010 to 2016")
    housing_plot.set_ylimit(min_units - std_units),(max_units + std_units)
    plt.close(figure_plot)
    return pn.pane.Matplotlib(figure_plot)

In [8]:
def average_sales_price():
  
    rent=sfo_data[['sale_price_sqr_foot', 'gross_rent']].groupby('year').mean()

    plt.plot(rent.index, rent['sale_price_sqr_foot'])
    plt.xlim(rent.index[0],rent.index[-1])
    plt.suptitle('Average Sale Price per Square Foot in San Francisco', weight='bold')
    plt.xlabel('Year')
    plt.ylabel('Avg. Sale Price')
    plt.show()
    plt.close()
    return pn.pane.hvplot.bar(rent.index, rent['sale_price_sqr_foot'])

In [9]:
def average_price_by_neighborhood():
   
    plt.plot(rent.index, rent['gross_rent'])
    plt.xlim(rent.index[0],rent.index[-1])
    plt.suptitle('Average Gross Rent in San Francisco', weight='bold')
    plt.xlabel('Year')
    plt.ylabel('Gross Rent')
    plt.show()
    plt.close()
    return pn.pane.hvplot.bar(rent.index, rent['gross_rent'])

In [10]:
def top_most_expensive_neighborhoods():
 
    san_fran_df = sfo_data.groupby(['year', 'neighborhood']).mean()
    
    most_expensive_neighborhoods = san_fran_df['sale_price_sqr_foot'].hvplot(xlabel='year', ylabel='average sale per square foot', 
    label=' ' ,groupby='neighborhood', height=300,width=450)
    return pn.pane.hvplot.bar(most_expensive_neighborhoods)

In [11]:
def price_per_square_root():
 
    top_10_most_expensive = sfo_data.groupby('neighborhood').mean().sort_values(ascending=False, by='sale_price_sqr_foot')

    price_per_square = top_10_most_expensive['sale_price_sqr_foot'].head(10).hvplot.bar(ylabel='average sale price per square foot',
    xlabel='neighborhood',label='top 10 most expensive neighborhoods',rot=35,height=375,width=625,)
    return pn.pane.hvplot.bar(price_per_square)

In [12]:
def parallel_categories():
    top_10_most_expensive = top_10_most_expensive.head(10).reset_index()
    top_10_most_expensive.head(10)

    comparative_maps = san_fran_df.head(10).hvplot.bar(x="year", y=["gross_rent","sale_price_sqr_foot"], xlabel= "Gross Rent Expense and Sale Price Per Sqr Foot", ylabel= "Dollar Amount", title="Top 10 Most Expensive Neighborhoods in San Francisco", groupby='neighborhood', height=400,rot=46)
    return pn.pane.hvplot.bar(comparative_maps)

In [13]:
def neighborhood_map():
    all_neighborhoods = sfo_data.groupby(by="neighborhood").mean()
    all_neighborhoods.reset_index(inplace=True)
    all_neighborhoods.rename(columns={"neighborhood": "Neighborhood"}, inplace=True)
    neighborhoods = pd.merge(df_neighborhood_locations, all_neighborhoods, on="Neighborhood")
    neighborhoods.head()

    px.set_mapbox_access_token('pk.eyJ1Ijoic2tvaG5sZTE0IiwiYSI6ImNrdXJwenVvNzU3cHkycG1hY2RzOGZydTIifQ.CWPJisDLmmjfABpv9Y2Kog')


    neighborhood_map_scatter = px.scatter_mapbox(neighborhoods,lat="Lat",lon="Lon",size="sale_price_sqr_foot",
    color="gross_rent", title="Average Sale Price per Square Foot and Gross Rent in San Francisco")
    return pn.pane.scatter_mapbox(neighborhood_map_scatter)

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

## Serve the Panel Dashboard

In [ ]:
# Create a Title for the Dashboard
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

plot_column = pn.Column(
    housing_units_per_year(), 
    average_gross_rent(),
    average_sales_price(),
    average_price_by_neighborhood(),
    top_most_expensive_neighborhoods(),
    price_per_square_foot())
        
parallel_column = pn.Column(parallel_categories())

map_column = pn.Column(neighborhood_map())

# Create tabs
panel = pn.Tabs(
    ("Plot", plot_column), 
    ("Parallel", parallel_column), 
    ("Map", map_column))

In [ ]:
# Serve the# dashboard
panel.servable()

# Debugging

Note: Some of the Plotly express plots may not render in the notebook through the panel functions.

However, you can test each plot by uncommenting the following code

In [ ]:
# housing_units_per_year()

In [ ]:
# average_gross_rent()

In [ ]:
# average_sales_price()

In [ ]:
# average_price_by_neighborhood()

In [ ]:
# top_most_expensive_neighborhoods()

In [ ]:
# most_expensive_neighborhoods_rent_sales()

In [ ]:
# neighborhood_map().show()

In [ ]:
# parallel_categories()

In [ ]:
# parallel_coordinates()

In [ ]:
# sunburst()